# 01 - Carregamento de Dados

Este notebook realiza:
1. Carregamento das configurações YAML
2. Carregamento de todos os arquivos de scouts
3. Carregamento da tabela de pesos
4. Validação da estrutura dos dados

In [ ]:
import pandas as pd
import yaml
from pathlib import Path
import os

# Configurar diretório base
BASE_DIR = Path("c:/jobs/botafogo/v3")
CONFIG_DIR = BASE_DIR / "config"
INPUTS_DIR = BASE_DIR / "bases" / "inputs"

## 1. Carregar Configurações

In [ ]:
# Carregar config.yaml
with open(CONFIG_DIR / "config.yaml", "r", encoding="utf-8") as f:
    config = yaml.safe_load(f)

# Carregar positions.yaml
with open(CONFIG_DIR / "positions.yaml", "r", encoding="utf-8") as f:
    positions_config = yaml.safe_load(f)

print("Configurações carregadas com sucesso!")
print(f"\nApp: {config['app']['name']} v{config['app']['version']}")

## 2. Carregar Arquivos de Scouts

In [ ]:
# Diretório dos scouts
SCOUTS_DIR = INPUTS_DIR / "scouts_base"

# Lista de arquivos de scouts
scout_files = list(SCOUTS_DIR.glob("*.xlsx"))
print(f"Arquivos encontrados: {len(scout_files)}")
for f in scout_files:
    print(f"  - {f.name}")

In [ ]:
# Carregar e concatenar todos os arquivos de scouts
dfs_scouts = []

for file_path in scout_files:
    print(f"Carregando {file_path.name}...")
    df = pd.read_excel(file_path)
    df["source_file"] = file_path.name  # Adiciona coluna de origem
    dfs_scouts.append(df)
    print(f"  -> {len(df)} jogadores, {len(df.columns)} colunas")

# Concatenar todos os DataFrames
df_scouts = pd.concat(dfs_scouts, ignore_index=True)
print(f"\nTotal: {len(df_scouts)} jogadores carregados")

In [ ]:
# Visualizar estrutura dos dados
print(f"Shape: {df_scouts.shape}")
print(f"\nPrimeiras colunas:")
print(df_scouts.columns[:20].tolist())

In [ ]:
# Verificar colunas de identificação
id_columns = ["player_id", "competition_id", "team_id", "primary_position", "player_season_most_recent_match"]
print("Colunas de identificação:")
for col in id_columns:
    if col in df_scouts.columns:
        print(f"  {col}: {df_scouts[col].nunique()} valores únicos")
    else:
        print(f"  {col}: NÃO ENCONTRADA")

In [ ]:
# Verificar posições disponíveis
print("\nPosições (primary_position):")
print(df_scouts["primary_position"].value_counts())

## 3. Carregar Tabela de Pesos

In [ ]:
# Carregar tabela de pesos
WEIGHTS_FILE = INPUTS_DIR / "business" / "base_peso.xlsx"
df_weights = pd.read_excel(WEIGHTS_FILE)

print(f"Tabela de pesos carregada: {df_weights.shape}")
print(f"\nColunas: {df_weights.columns.tolist()}")

In [ ]:
# Verificar estrutura da tabela de pesos
print("Primeiras linhas:")
df_weights.head()

In [ ]:
# Filtrar indicadores a considerar
df_weights_active = df_weights[df_weights["CONSIDERAR?"] == "SIM"].copy()
print(f"Indicadores ativos (CONSIDERAR? = SIM): {len(df_weights_active)}")
print(f"Indicadores ignorados: {len(df_weights) - len(df_weights_active)}")

In [ ]:
# Verificar classificações
print("\nCLASSIFICACAO RANKING:")
print(df_weights_active["CLASSIFICACAO RANKING"].value_counts())

print("\nMelhor para:")
print(df_weights_active["Melhor para"].value_counts())

In [ ]:
# Colunas de posições na tabela de pesos
position_columns = ["GK", "RCB", "LCB", "CB", "RB", "LB", "DM", "CM", "AM", "LW", "RW", "CF"]
print("\nColunas de posições disponíveis:")
for col in position_columns:
    if col in df_weights.columns:
        print(f"  {col}: OK")
    else:
        print(f"  {col}: NÃO ENCONTRADA")

## 4. Validação dos Dados

In [ ]:
# Verificar se os indicadores da tabela de pesos existem nos scouts
indicadores_pesos = set(df_weights_active["INDICADOR"].str.strip())
colunas_scouts = set(df_scouts.columns)

# Indicadores que existem em ambos
indicadores_match = indicadores_pesos.intersection(colunas_scouts)
indicadores_missing = indicadores_pesos - colunas_scouts

print(f"Indicadores na tabela de pesos: {len(indicadores_pesos)}")
print(f"Indicadores encontrados nos scouts: {len(indicadores_match)}")
print(f"Indicadores não encontrados: {len(indicadores_missing)}")

if indicadores_missing:
    print(f"\nIndicadores faltantes (primeiros 10):")
    for ind in list(indicadores_missing)[:10]:
        print(f"  - {ind}")

In [ ]:
# Resumo do carregamento
print("=" * 50)
print("RESUMO DO CARREGAMENTO")
print("=" * 50)
print(f"Total de jogadores: {len(df_scouts)}")
print(f"Total de colunas nos scouts: {len(df_scouts.columns)}")
print(f"Arquivos de scouts: {len(scout_files)}")
print(f"Indicadores ativos: {len(df_weights_active)}")
print(f"Indicadores correspondentes: {len(indicadores_match)}")
print("=" * 50)

## 5. Salvar Dados Carregados

In [ ]:
# Salvar dados intermediários para os próximos notebooks
OUTPUT_DIR = BASE_DIR / "bases" / "outputs"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Converter colunas problemáticas antes de salvar
# Colunas string que devem permanecer como string (mas manter NaN como NaN, não converter para "nan")
string_cols = ['season_name', 'competition_name', 'team_name', 'player_name', 
               'player_first_name', 'player_last_name', 'player_known_name',
               'primary_position', 'source_file']

for col in string_cols:
    if col in df_scouts.columns:
        # Converter apenas valores não-nulos para string
        mask = df_scouts[col].notna()
        df_scouts.loc[mask, col] = df_scouts.loc[mask, col].astype(str)

# Salvar scouts consolidados
df_scouts.to_parquet(OUTPUT_DIR / "_temp_scouts_raw.parquet", index=False)
print(f"Scouts salvos: {OUTPUT_DIR / '_temp_scouts_raw.parquet'}")

# Converter colunas da tabela de pesos para string também
weights_string_cols = ['INDICADOR', 'CLASSIFICACAO RANKING', 'SUBCLASSIFICACAO RANKING', 
                       'CONSIDERAR?', 'ESPECIAL?', 'Melhor para', 'tipo_agreg', 
                       'Explicação indicador']

for col in weights_string_cols:
    if col in df_weights_active.columns:
        # Converter apenas valores não-nulos para string
        mask = df_weights_active[col].notna()
        df_weights_active.loc[mask, col] = df_weights_active.loc[mask, col].astype(str)

# Salvar pesos ativos
df_weights_active.to_parquet(OUTPUT_DIR / "_temp_weights_active.parquet", index=False)
print(f"Pesos salvos: {OUTPUT_DIR / '_temp_weights_active.parquet'}")

print("\nDados prontos para a próxima etapa!")